In [1]:
# Dependencies and Setup mike
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Name of the CSV file
file = "Resources/cal_fire.csv"

In [3]:
# read the CSV in pandas
df = pd.read_csv(file)

In [4]:
# Preview of the DataFrame
df.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,True,2018-01-09 13:46:00,2017-10-31 11:22:00,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37,100,...,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09 13:46:00,2018-01-09,2017-10-31,N,False,False
1,River Fire,True,2013-02-28 20:00:00,2013-02-24 08:16:00,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",406,100,...,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28 20:00:00,2013-02-28,2013-02-24,N,True,False
2,Fawnskin Fire,True,2013-04-22 09:00:00,2013-04-20 17:30:00,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30,100,...,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22 09:00:00,2013-04-22,2013-04-20,N,False,False
3,Gold Fire,True,2013-05-01 07:00:00,2013-04-30 12:59:00,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274,100,...,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01 07:00:00,2013-05-01,2013-04-30,N,True,False
4,Panther Fire,True,2013-05-09 09:00:00,2013-05-01 09:12:00,CAL FIRE Butte Unit,NaN,Tehama,"140K3 Line, 8 miles northwest of Butte Meadows",6965,100,...,40.190062,NaN,53122f0f-fefc-4dbf-b2d8-566b42ced66d,https://www.fire.ca.gov/incidents/2013/5/1/pan...,2013-05-09 09:00:00,2013-05-09,2013-05-01,N,True,False


In [5]:
# Reduce dataframe to only relevant columns
reduced_df = df.loc[:, ["incident_name", "incident_date_created", "incident_county",
                                    "incident_acres_burned", "incident_longitude", 
                                    "incident_latitude", "incident_date_extinguished"]]
reduced_df

,incident_name,incident_date_created,incident_county,incident_acres_burned,incident_longitude,incident_latitude,incident_date_extinguished
0,Bridge Fire,2017-10-31 11:22:00,Shasta,37,-122.309000,40.774000,2018-01-09 13:46:00
1,River Fire,2013-02-24 08:16:00,Inyo,406,-118.016510,36.602575,2013-02-28 20:00:00
2,Fawnskin Fire,2013-04-20 17:30:00,San Bernardino,30,-116.941311,34.288877,2013-04-22 09:00:00
3,Gold Fire,2013-04-30 12:59:00,Madera,274,-119.635004,37.116295,2013-05-01 07:00:00
4,Panther Fire,2013-05-01 09:12:00,Tehama,6965,-121.595555,40.190062,2013-05-09 09:00:00
...,...,...,...,...,...,...,...
1524,Cornetts Fire,2020-06-13 14:27:51,Mariposa,191,-120.085220,37.408270,NaN
1525,Dawson Fire,2020-06-13 18:32:06,Riverside,10,-117.461925,33.776033,NaN
1526,Hopland Fire,2020-06-13 18:52:54,Mendocino,34,-80.491630,39.147830,2020-06-14 07:36:00
1527,Drum Fire,2020-06-14 15:03:45,Santa Barbara,600,-120.288670,34.633090,NaN


In [9]:
# Change column names to: Fire Name, Start Date, County, Acres Burned, Longitude, Latitude, Extinguished Date
print(reduced_df.rename(columns={'incident_name':'Fire Name', 'incident_date_created':'Start Date', 'incident_county':'County'
                        'incident_acres_burned':'Acres Burned', 'incident_longitude': 'Longitude', 
                                    'incident_latitude':'Latitude', 'incident_date_extinguished':'Extinguished Date'}))

SyntaxError: invalid syntax (<ipython-input-9-05194d5ecbe2>, line 3)

In [ ]:
# Eliminate rows without data

In [ ]:
# Check data types
reduced_df.dtypes

In [ ]:
# Create a Jupyter Notebook illustrating the final data analysis

In [ ]:
# Where does it rain most frequently in California and is there a correlation with wildfires?

In [ ]:
# Is there a correlation between total amount of rain in a specific area and the total number of wildfires?

In [ ]:
# Is there a correlation between total number of acres burned and precipitation or the size of wildfires?

In [ ]:
# Is there a correlation between the duration of a wildfire and precipitation?

In [ ]:
# Do fires last longer when there is less precipitation? Could there be outside factors shortening the duration of a wildfire?